In [ ]:
# Mounting the google drive for importing the images
from google.colab import drive

In [ ]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.1 MB 7.8 MB/s 
     |████████████████████████████████| 235 kB 38.2 MB/s 
     |████████████████████████████████| 4.7 MB 37.9 MB/s 
     |████████████████████████████████| 164 kB 54.7 MB/s 
     |████████████████████████████████| 181 kB 52.6 MB/s 
     |████████████████████████████████| 78 kB 6.5 MB/s 
     |████████████████████████████████| 63 kB 1.2 MB/s 
     |████████████████████████████████| 51 kB 5.8 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=f074227db301fc7e1c5c33784ec0fe058e7600edb00a15d1dca199af5c5c64e3
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 7.5 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=3f365f15a7689709c4cfd33e6c4ac3b3873816c3dfdd0311e9d7f50a3ebd1ef8
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle
from sklearn.neighbors import NearestNeighbors
import cv2
from google.colab.patches import cv2_imshow
import streamlit as st
from PIL import Image
from pyngrok import ngrok
import streamlit as st

In [ ]:
%%writefile app.py
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle
from sklearn.neighbors import NearestNeighbors
import cv2
from google.colab.patches import cv2_imshow
import streamlit as st
from PIL import Image
from pyngrok import ngrok
import streamlit as st
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

hide_menu_style ="""
          <style>
          #MainMenu {visibility : hidden ; }
          footer {visibility : hidden ; }
          </style>
          """
st.markdown(hide_menu_style, unsafe_allow_html=True)

model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False

model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result

filenames = []

for file in os.listdir('/content/drive/MyDrive/Train_images'):
    filenames.append(os.path.join('/content/drive/MyDrive/Train_images',file))

feature_list = []

for file in tqdm(filenames):
    feature_list.append(extract_features(file,model))

pickle.dump(feature_list,open('embeddings.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))


feature_list = np.array(pickle.load(open('/content/embeddings.pkl','rb')))
filenames = pickle.load(open('/content/filenames.pkl','rb'))


def feature_extraction(img_path,model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)

    return normalized_result

def recommend(features,feature_list):
    neighbors = NearestNeighbors(n_neighbors=6
    , algorithm='brute', metric='euclidean')
    neighbors.fit(feature_list)

    distances, indices = neighbors.kneighbors([features])

    return indices

original_title = '<p style="font-family:Bodoni MT Black; color:Black;text-align: center; font-size: 35px;"><strong>Pesonalised Fashion Recommendention System</strong></p>'
st.markdown(original_title, unsafe_allow_html=True)

def add_bg_from_url():
    st.markdown(
         f"""
         <style>
         .stApp {{
             background-image: url("https://cdn.pixabay.com/photo/2016/11/22/21/57/apparel-1850804_1280.jpg");
             background-attachment: fixed;
             background-size: cover;
         }}
         </style>
         """,
         unsafe_allow_html=True
     )

add_bg_from_url() 

def save_uploaded_file(uploaded_file):
    try:
        with open(os.path.join('/content/Up',uploaded_file.name),'wb') as f:
            f.write(uploaded_file.getbuffer())
        return 1
    except:
        return 0

uploaded_file = st.file_uploader("Choose an image to search")
if uploaded_file is not None:
    if save_uploaded_file(uploaded_file):
        # display the file
        display_image = Image.open(uploaded_file)
        st.image(display_image,width = 300,use_column_width=300, output_format="auto",caption ="Uploaded Image")
        # feature extract
        features = feature_extraction(os.path.join("/content/Up",uploaded_file.name),model)
        #st.text(features)
        # recommendention
        indices = recommend(features,feature_list)
        # show
        col1,col2,col3,col4,col5 = st.columns(5)

        with col1:
            st.image(filenames[indices[0][0]])
        with col2:
            st.image(filenames[indices[0][1]])
        with col3:
            st.image(filenames[indices[0][2]])
        with col4:
            st.image(filenames[indices[0][3]])
        with col5:
            st.image(filenames[indices[0][4]])
        
    else:
        st.header("Some error occured in file upload")

Overwriting app.py


In [ ]:
ngrok.set_auth_token("2Dk41YkP0EZZI4KUbnnO7tnZXuq_5rHzfpPvho1v1D1gozYZq")

INFO:pyngrok.process:Updating authtoken for default "config_path" of "ngrok_path": /usr/local/lib/python3.7/dist-packages/pyngrok/bin/ngrok
2022-08-31 12:52:13.164 INFO    pyngrok.process: Updating authtoken for default "config_path" of "ngrok_path": /usr/local/lib/python3.7/dist-packages/pyngrok/bin/ngrok


In [ ]:
!nohup streamlit run app.py --server.port 80 &
url = ngrok.connect(port = '80')
print(url)

nohup: appending output to 'nohup.out'


INFO:pyngrok.ngrok:Opening tunnel named: http-80-ce8a9829-a802-42e1-887a-0cd792388fe6
2022-08-31 12:52:17.514 INFO    pyngrok.ngrok: Opening tunnel named: http-80-ce8a9829-a802-42e1-887a-0cd792388fe6
INFO:pyngrok.process.ngrok:t=2022-08-31T12:52:17+0000 lvl=info msg="no configuration paths supplied"
2022-08-31 12:52:17.632 INFO    pyngrok.process.ngrok: t=2022-08-31T12:52:17+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2022-08-31T12:52:17+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2022-08-31 12:52:17.645 INFO    pyngrok.process.ngrok: t=2022-08-31T12:52:17+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
INFO:pyngrok.process.ngrok:t=2022-08-31T12:52:17+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2022-08-31 12:52:17.655 INFO    pyngrok.process.ngrok: t=2022-08-31T12:52:17+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngro

NgrokTunnel: "http://369b-35-247-111-236.ngrok.io" -> "http://localhost:80"


2022-08-31 12:52:18.065 INFO    pyngrok.process.ngrok: t=2022-08-31T12:52:18+0000 lvl=info msg="started tunnel" obj=tunnels name="http-80-ce8a9829-a802-42e1-887a-0cd792388fe6 (http)" addr=http://localhost:80 url=http://369b-35-247-111-236.ngrok.io
INFO:pyngrok.process.ngrok:t=2022-08-31T12:52:18+0000 lvl=info msg="started tunnel" obj=tunnels name=http-80-ce8a9829-a802-42e1-887a-0cd792388fe6 addr=http://localhost:80 url=https://369b-35-247-111-236.ngrok.io
2022-08-31 12:52:18.076 INFO    pyngrok.process.ngrok: t=2022-08-31T12:52:18+0000 lvl=info msg="started tunnel" obj=tunnels name=http-80-ce8a9829-a802-42e1-887a-0cd792388fe6 addr=http://localhost:80 url=https://369b-35-247-111-236.ngrok.io
INFO:pyngrok.process.ngrok:t=2022-08-31T12:52:18+0000 lvl=info msg=end pg=/api/tunnels id=1fdf23addf7c6d5a status=201 dur=152.783435ms
2022-08-31 12:52:18.097 INFO    pyngrok.process.ngrok: t=2022-08-31T12:52:18+0000 lvl=info msg=end pg=/api/tunnels id=1fdf23addf7c6d5a status=201 dur=152.783435ms
IN